In [1]:
from scapy.all import rdpcap, IP, TCP, UDP, Raw, IPv6

In [2]:
# Ensure the PCAP file exists before reading
pcap_file = "0.pcap"
try:
    packets = rdpcap(pcap_file)
except FileNotFoundError:
    print(f"File {pcap_file} not found")

In [3]:
ims_ips = ["14.139.98.79", "10.0.137.79", "64:ff9b::e8b:624f"]  # From nslookup.com and by trying ping

In [4]:
port_packets, ims_packets = [], []
super_users = 0
for packet in packets:
    IPv = IP if IP in packet else IPv6 if IPv6 in packet else None
    if IPv and IPv in packet:
        src_ip = packet[IPv].src
        dst_ip = packet[IPv].dst
        if dst_ip in ims_ips:
            ims_packets.append(packet)
        
        protocol = TCP if TCP in packet else UDP if UDP in packet else None
        if protocol and (packet[protocol].sport == 4321 or packet[protocol].dport == 4321):
            port_packets.append(packet)
        
        if Raw in packet:
            payload = packet[Raw].load.decode(errors='ignore')
            super_users += payload.lower().count("superuser") > 0

In [5]:
port_packets

[<Ether  dst=2c:41:38:1d:e9:40 src=00:90:7f:3e:02:d0 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=103 id=17453 flags= frag=0 ttl=126 proto=udp chksum=0x9996 src=172.16.128.169 dst=172.16.133.248 |<UDP  sport=4321 dport=snmp len=83 chksum=0x9082 |<SNMP  version='v1' 0x0 <ASN1_INTEGER[0]> community=<ASN1_STRING[b'public']> PDU=<SNMPget  id=0x491c2f5 <ASN1_INTEGER[76661493]> error='no_error' 0x0 <ASN1_INTEGER[0]> error_index=0x0 <ASN1_INTEGER[0]> varbindlist=[<SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.16.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.1.3.0']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>] |> |>>>>,
 <Ether  dst=00:90:7f:3e:02:d0 src=2c:41:38:1d:e9:40 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=115 id=62980 flags= frag=

In [6]:
sum([len(packet) for packet in port_packets])

246

In [7]:
port_packets[0].show2()

###[ Ethernet ]###
  dst       = 2c:41:38:1d:e9:40
  src       = 00:90:7f:3e:02:d0
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 103
     id        = 17453
     flags     = 
     frag      = 0
     ttl       = 126
     proto     = udp
     chksum    = 0x9996
     src       = 172.16.128.169
     dst       = 172.16.133.248
     \options   \
###[ UDP ]###
        sport     = 4321
        dport     = snmp
        len       = 83
        chksum    = 0x9082
###[ SNMP ]###
           version   = 'v1' 0x0 <ASN1_INTEGER[0]>
           community = <ASN1_STRING[b'public']>
           \PDU       \
            |###[ SNMPget ]###
            |  id        = 0x491c2f5 <ASN1_INTEGER[76661493]>
            |  error     = 'no_error' 0x0 <ASN1_INTEGER[0]>
            |  error_index= 0x0 <ASN1_INTEGER[0]>
            |  \varbindlist\
            |   |###[ SNMPvarbind ]###
            |   |  oid       = <ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']>
     

In [8]:
# Print Results
print(f"Q1: Unique connections to IMS server: {len(ims_packets)}")
print(f"Q3: Total data transferred on port 4321: {sum(len(pkt) for pkt in port_packets)} bytes")
print(f"Q4: Total number of SuperUsers: {super_users}")

Q1: Unique connections to IMS server: 0
Q3: Total data transferred on port 4321: 246 bytes
Q4: Total number of SuperUsers: 0


Since the Connections to IMS server are `0` therefore, for Q2 (find the registered course) the registered course can't be found without having any packet sent to IMS.